
#  광고 클릭율 예측을 위한 모델 개발

- **대용량 데이터**  
 대규모 데이터셋으로 구성이 되어 있어서 duckdb 라이브러리로 데이터를 랜덤 추출해서 샘플링 진행
   ->처음 샘플링 단계에서 데이터를 제대로 컨트롤 하지 못하여 모델의 성능이 예상보다 낮게 나온것 같다
   -> 대규모 데이터를 다룰 수 있는 방법을 좀 더 연구해서 다시 모델 학습을 진행
   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00


In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

## 데이터 연결

## Chunk Data
메모리 이슈로 인해 전체 데이터를 로드하기 어려워서 chunk형태로 데이터 로드

In [5]:
# CSV 파일 경로
csv_path = '/content/drive/MyDrive/zerobase/광고클릭률예측/open/'
# 청크 크기 설정
chunksize = 100000

# CSV 파일을 청크 단위로 읽어오는 반복자(iterator)를 생성
chunk_iter = pd.read_csv(csv_path + 'train.csv', chunksize=chunksize)

# 청크를 하나씩 처리하고 저장
for i, chunk in enumerate(chunk_iter):
    # 청크의 처리 작업 예시: 데이터 확인
    print(f"Processing chunk {i+1}")

    # 처리된 청크 데이터를 파일로 저장
    output_file = f'processed_chunk_{i+1}.csv'
    chunk.to_csv(output_file, index=False)

    print(f"Saved chunk {i+1} to {output_file}")

Processing chunk 1
Saved chunk 1 to processed_chunk_1.csv
Processing chunk 2
Saved chunk 2 to processed_chunk_2.csv
Processing chunk 3
Saved chunk 3 to processed_chunk_3.csv
Processing chunk 4
Saved chunk 4 to processed_chunk_4.csv
Processing chunk 5
Saved chunk 5 to processed_chunk_5.csv
Processing chunk 6
Saved chunk 6 to processed_chunk_6.csv
Processing chunk 7
Saved chunk 7 to processed_chunk_7.csv
Processing chunk 8
Saved chunk 8 to processed_chunk_8.csv
Processing chunk 9
Saved chunk 9 to processed_chunk_9.csv
Processing chunk 10
Saved chunk 10 to processed_chunk_10.csv
Processing chunk 11
Saved chunk 11 to processed_chunk_11.csv
Processing chunk 12
Saved chunk 12 to processed_chunk_12.csv
Processing chunk 13
Saved chunk 13 to processed_chunk_13.csv
Processing chunk 14
Saved chunk 14 to processed_chunk_14.csv
Processing chunk 15
Saved chunk 15 to processed_chunk_15.csv
Processing chunk 16
Saved chunk 16 to processed_chunk_16.csv
Processing chunk 17
Saved chunk 17 to processed_chu

In [6]:
import glob

# 청크 파일이 저장된 디렉토리 경로
directory_path = '/content/'

# 디렉토리 내에서 특정 패턴에 맞는 모든 파일 경로를 가져오기
chunk_files = sorted(glob.glob(directory_path + 'processed_chunk_*.csv'))

# 가져온 파일 리스트 출력
print(f"Found {len(chunk_files)} chunk files.")

Found 287 chunk files.


In [6]:
test = pd.read_csv('/content/drive/MyDrive/zerobase/광고클릭률예측/open/test.csv')

In [7]:
# 청크 파일이 저장된 디렉토리 경로
directory_path = '/content/'
chunksize = 100000

# 청크 파일 리스트 가져오기
chunk_files = sorted(glob.glob(directory_path + 'processed_chunk_*.csv'))

# 청크 파일에서 범주형 열 추출
def extract_categorical_columns(file):
    try:
        # 청크 파일 읽기
        chunk = pd.read_csv(file, nrows=1)  # 첫 번째 행만 읽어 열 이름을 추출
        categorical_cols = chunk.select_dtypes(include=['object']).columns
        return categorical_cols
    except Exception as e:
        print(f"Error processing file {file}: {e}")
        return []

# 첫 번째 청크 파일에서 범주형 열 추출 (데이터를 기준으로 반복처리)
example_file = chunk_files[0]  # 예를 들어 첫 번째 파일 사용
categorical_columns = extract_categorical_columns(example_file)

print("Categorical columns in the first chunk file:", categorical_columns)

test_categorical_columns = test.select_dtypes(include=['object']).columns
print(test_categorical_columns)
#print(train_categorical_columns)
#test.isnull().sum()

Categorical columns in the first chunk file: Index(['ID', 'F01', 'F02', 'F03', 'F05', 'F07', 'F08', 'F09', 'F10', 'F12',
       'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F22', 'F23', 'F25', 'F26',
       'F28', 'F30', 'F31', 'F34', 'F35', 'F37', 'F39'],
      dtype='object')
Index(['ID', 'F01', 'F02', 'F03', 'F05', 'F07', 'F08', 'F09', 'F10', 'F12',
       'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F22', 'F23', 'F25', 'F26',
       'F28', 'F30', 'F31', 'F34', 'F35', 'F37', 'F39'],
      dtype='object')


##기본 통계 및 요약 정보

In [ ]:
# 청크 파일이 저장된 디렉토리 경로
directory_path = '/content/'

# 디렉토리 내에서 특정 패턴에 맞는 모든 파일 경로를 가져오기
chunk_files = sorted(glob.glob(directory_path + 'processed_chunk_*.csv'))

# 가져온 파일 리스트 출력
print(f"Found {len(chunk_files)} chunk files.")

# 각 청크 파일을 읽어서 통계 정보 출력
for file in chunk_files:
    # 청크 파일 읽기
    chunk = pd.read_csv(file)

    # 데이터 기본 통계 출력
    print(f"Statistics for {file}:")

    # 데이터프레임 정보 출력
    print("DataFrame Info:")
    print(chunk.info())

    # 기초 통계 요약 출력 (수치형 열에 대해)
    print("\nDescriptive Statistics:")
    print(chunk.describe(include='all'))  # 모든 데이터 타입의 요약 통계

    # 결측치 개수 확인
    print("\nMissing Values:")
    print(chunk.isnull().sum())

    # 데이터 타입 확인
    print("\nData Types:")
    print(chunk.dtypes)

    # 청크의 첫 5개 행 출력
    print("\nFirst 5 rows:")
    print(chunk.head())

    print("\n" + "="*50 + "\n")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
freq      54512           NaN           NaN    21737    89277           NaN   
mean        NaN  1.921111e+04      1.806516      NaN      NaN      8.408840   
std         NaN  7.039625e+04      7.141144      NaN      NaN     40.766531   
min         NaN  0.000000e+00      0.000000      NaN      NaN      0.000000   
25%         NaN  4.470000e+02      0.000000      NaN      NaN      2.000000   
50%         NaN  2.741000e+03      0.000000      NaN      NaN      4.000000   
75%         NaN  1.015050e+04      1.000000      NaN      NaN      9.000000   
max         NaN  2.112422e+06    305.000000      NaN      NaN   7583.000000   

            F37           F38      F39  
count    100000  96585.000000   100000  
unique     3413           NaN     4713  
top     ZVSTLNM           NaN  GVRZOHG  
freq       6516           NaN     3007  
mean        NaN      0.190961      NaN  
std         NaN      2.029145      NaN  
min         NaN      0.000000      NaN  
25%

# 데이터 전처리

## 결측치 제거

수치형데이터 결측치는 0으로 </br>
범주형데이터 결측치는 ''Missing''으로 대체

### chunk data 결측치 제거

In [7]:
import pandas as pd
import glob
import os


# 청크 파일이 저장된 디렉토리 경로
directory_path = '/content/'
chunksize = 100000

# 청크 파일 리스트 가져오기
chunk_files = sorted(glob.glob(directory_path + 'processed_chunk_*.csv'))

# 처리된 청크 데이터를 저장할 디렉토리
processed_dir = '/content/processed_chunks_cleaned/'
os.makedirs(processed_dir, exist_ok=True)

# 처리되지 않은 파일 목록 가져오기
def get_unprocessed_files(chunk_files, processed_dir):
    processed_files = set(os.listdir(processed_dir))
    unprocessed_files = [file for file in chunk_files if os.path.basename(file) not in processed_files]
    return unprocessed_files

# 처리되지 않은 파일 목록 가져오기
unprocessed_files = get_unprocessed_files(chunk_files, processed_dir)

# 각 청크 파일을 읽어서 처리 후 저장
for file in unprocessed_files:
    # 청크 파일 읽기
    chunk = pd.read_csv(file)

    # 수치형 데이터 결측치 0으로 대체
    numeric_cols = chunk.select_dtypes(include=['number']).columns
    chunk[numeric_cols] = chunk[numeric_cols].fillna(0)

    # 범주형 데이터 결측치 'Missing'으로 대체
    categorical_cols = chunk.select_dtypes(include=['object']).columns
    chunk[categorical_cols] = chunk[categorical_cols].fillna('Missing')

    # 처리된 청크를 새로운 파일로 저장
    output_file = os.path.join(processed_dir, os.path.basename(file).replace('processed_chunk_', 'processed_chunk_cleaned_'))
    chunk.to_csv(output_file, index=False)
    print(f"Processed and saved {file} to {output_file}")

print(f"Processed {len(unprocessed_files)} chunks.")

Processed and saved /content/processed_chunk_1.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_1.csv
Processed and saved /content/processed_chunk_10.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_10.csv
Processed and saved /content/processed_chunk_100.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_100.csv
Processed and saved /content/processed_chunk_101.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_101.csv
Processed and saved /content/processed_chunk_102.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_102.csv
Processed and saved /content/processed_chunk_103.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_103.csv
Processed and saved /content/processed_chunk_104.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_104.csv
Processed and saved /content/processed_chunk_105.csv to /content/processed_chunks_cleaned/processed_chunk_cleaned_105.csv
Processed and saved /content/p

### test data 결측치 제거


In [9]:
import pandas as pd


# 수치형 데이터 결측치 0으로 대체
numeric_cols = test.select_dtypes(include=['number']).columns
test[numeric_cols] = test[numeric_cols].fillna(0)

# 범주형 데이터 결측치 'Missing'으로 대체
categorical_cols = test.select_dtypes(include=['object']).columns
test[categorical_cols] = test[categorical_cols].fillna('Missing')

# 처리된 데이터프레임을 새로운 파일로 저장
output_file_path = '/content/test_cleaned.csv'
test.to_csv(output_file_path, index=False)

print("Data cleaning completed and saved to", output_file_path)

Data cleaning completed and saved to /content/test_cleaned.csv


In [10]:
test = pd.read_csv('/content/test_cleaned.csv')
test.head()

,ID,F01,F02,F03,F04,F05,F06,F07,F08,F09,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,TEST_0000000,Missing,Missing,IAGJDOH,0.0,Missing,-1,WBNHHNS,MIGYEEG,NENNAGQ,...,MFPUCBU,GTISJWW,1144.0,2.0,Missing,IRUDRFB,0.0,VINCJKA,0.0,AWWBMDJ
1,TEST_0000001,SDVLQID,VDDLZCR,DTZFPRW,1.0,KJLEQJC,3,OWLMRBR,MIGYEEG,UKFZEAD,...,NZGEZLW,GTISJWW,44364.0,0.0,ORGHLQN,IRUDRFB,0.0,KSGNDBR,0.0,OJTGSWO
2,TEST_0000002,XXQINZY,AXQDKZD,IAGJDOH,12.0,NFKKMGD,2,ILCWUTM,LPYPUNA,NENNAGQ,...,VHXETCF,GTISJWW,79.0,1.0,WRRYBLO,IRUDRFB,0.0,VINCJKA,0.0,INBFMRT
3,TEST_0000003,JCDXFYU,PILDDJU,IAGJDOH,22.0,LFPUEOV,209,HUHNWCJ,FTPHMPQ,FPCZMEO,...,NZGEZLW,KHZNEZF,44.0,9.0,QMOULXS,IRUDRFB,8.0,RWQBLLR,0.0,OYSGPBR
4,TEST_0000004,DZPQPXK,HKSCOJF,Missing,2.0,FMKTHYA,2,CFQLINN,FTPHMPQ,YOPLVLN,...,FGOVFJM,WCPHYPZ,1.0,2.0,WZAQBGE,IRUDRFB,0.0,XJLBFMB,1.0,OYSGPBR


In [11]:
# TEST 데이터 결측치 제거 확인
test.isnull().sum()

,0
ID,0
F01,0
F02,0
F03,0
F04,0
F05,0
F06,0
F07,0
F08,0
F09,0


In [11]:
# train데이터 결측치 확인

# 청크 파일이 저장된 디렉토리 경로
directory_path =  '/content/processed_chunks_cleaned/'
chunksize = 100000

# 청크 파일 리스트 가져오기
chunk_files = sorted(glob.glob(directory_path + 'processed_chunk_cleaned_*.csv'))

# 각 청크 파일을 읽어서 결측치 확인
for file in chunk_files:
    print(f"Processing file: {file}")

    # 청크 파일 읽기
    chunk = pd.read_csv(file, chunksize=chunksize)

    # 각 청크에 대해 결측치 확인
    for i, chunk_part in enumerate(chunk):
        print(f"Chunk {i+1} of {file}:")
        print(chunk_part.isnull().sum())  # 각 열의 결측치 개수 출력
        print()  # 빈 줄로 구분

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
F37      0
F38      0
F39      0
dtype: int64

Processing file: /content/processed_chunks_cleaned/processed_chunk_cleaned_258.csv
Chunk 1 of /content/processed_chunks_cleaned/processed_chunk_cleaned_258.csv:
ID       0
Click    0
F01      0
F02      0
F03      0
F04      0
F05      0
F06      0
F07      0
F08      0
F09      0
F10      0
F11      0
F12      0
F13      0
F14      0
F15      0
F16      0
F17      0
F18      0
F19      0
F20      0
F21      0
F22      0
F23      0
F24      0
F25      0
F26      0
F27      0
F28      0
F29      0
F30      0
F31      0
F32      0
F33      0
F34      0
F35      0
F36      0
F37      0
F38      0
F39      0
dtype: int64

Processing file: /content/processed_chunks_cleaned/processed_chunk_cleaned_259.csv
Chunk 1 of /content/processed_chunks_cleaned/processed_chunk_cleaned_259.csv:
ID       0
Click    0
F01      0
F02      0
F03      0
F04      0
F05      0
F06      0
F07      0
F08      0
F09      0
F10      

## 범주형 변수 인코딩

### train data 인코딩

In [ ]:
categorical_columns = test.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['ID',
 'F01',
 'F02',
 'F03',
 'F05',
 'F07',
 'F08',
 'F09',
 'F10',
 'F12',
 'F13',
 'F15',
 'F16',
 'F17',
 'F20',
 'F21',
 'F22',
 'F23',
 'F25',
 'F26',
 'F28',
 'F30',
 'F31',
 'F34',
 'F35',
 'F37',
 'F39']

In [8]:
import pandas as pd
import os
import glob
import pickle

# 청크 파일이 저장된 디렉토리 경로
directory_path =  '/content/processed_chunks_cleaned/'
chunksize = 100000

# 청크 파일 리스트 가져오기
chunk_files = sorted(glob.glob(directory_path + 'processed_chunk_cleaned_*.csv'))

# 처리된 청크를 저장할 디렉토리
encoded_dir = '/content/encoded_chunks/'
os.makedirs(encoded_dir, exist_ok=True)

# 범주형 열 정의
categorical_columns = [ 'F01', 'F02', 'F03', 'F05', 'F07', 'F08', 'F09', 'F10', 'F12', 'F13', 'F15', 'F16', 'F17', 'F20',
                                      'F21', 'F22', 'F23', 'F25', 'F26', 'F28', 'F30', 'F31', 'F34', 'F35', 'F37', 'F39']

# 범주형 열의 맵핑을 저장할 딕셔너리
mapping = {}

# 각 청크 파일을 읽어서 인코딩 후 저장
for file in chunk_files:
    try:
        # 청크 파일 읽기
        chunk = pd.read_csv(file)

        # 각 청크에 대해 Categorical 변환 및 맵핑 저장
        for col in categorical_columns:
            if col not in mapping:
                # 새로운 범주형 열의 맵핑 생성
                chunk[col] = pd.Categorical(chunk[col].astype(str))
                mapping[col] = dict(enumerate(chunk[col].cat.categories))
            else:
                # 이미 존재하는 맵핑을 기반으로 카테고리 재설정
                chunk[col] = pd.Categorical(chunk[col].astype(str), categories=list(mapping[col].values()))

            # 인코딩된 코드를 사용하여 열을 업데이트
            chunk[col] = chunk[col].cat.codes.astype('int')  # Ensure the type is int

        # 인코딩된 청크를 새로운 파일로 저장
        output_file = os.path.join(encoded_dir, os.path.basename(file).replace('processed_chunk_cleaned_', 'encoded_chunk_'))
        chunk.to_csv(output_file, index=False)
        print(f"Encoded and saved {file} to {output_file}")

    except Exception as e:
        print(f"Error processing file {file}: {e}")

# 맵핑을 파일로 저장
with open('/content/encoded_chunks/mapping.pkl', 'wb') as f:
    pickle.dump(mapping, f)

Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_1.csv to /content/encoded_chunks/encoded_chunk_1.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_10.csv to /content/encoded_chunks/encoded_chunk_10.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_100.csv to /content/encoded_chunks/encoded_chunk_100.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_101.csv to /content/encoded_chunks/encoded_chunk_101.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_102.csv to /content/encoded_chunks/encoded_chunk_102.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_103.csv to /content/encoded_chunks/encoded_chunk_103.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned_104.csv to /content/encoded_chunks/encoded_chunk_104.csv
Encoded and saved /content/processed_chunks_cleaned/processed_chunk_cleaned

### test data 인코딩

In [12]:
import pandas as pd
import os
import pickle

# 인코딩된 테스트 데이터를 저장할 파일 경로
encoded_test_file = '/content/encoded_chunks/encoded_test_data.csv'

# 저장된 맵핑 불러오기
with open('/content/encoded_chunks/mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)

# 테스트 데이터 읽기
test = pd.read_csv('/content/test_cleaned.csv')

# 테스트 데이터 인코딩
categorical_columns = [ 'F01', 'F02', 'F03', 'F05', 'F07', 'F08', 'F09', 'F10', 'F12', 'F13', 'F15', 'F16', 'F17', 'F20',
                                      'F21', 'F22', 'F23', 'F25', 'F26', 'F28', 'F30', 'F31', 'F34', 'F35', 'F37', 'F39']

for col in categorical_columns:
    if col in test.columns:
        # 테스트 데이터의 각 열을 학습된 범주로 인코딩
        test[col] = test[col].map(mapping[col])

# 인코딩된 테스트 데이터를 저장
test.to_csv(encoded_test_file, index=False)
print(f"Encoded test data saved to {encoded_test_file}")
print(f"Saving to: {encoded_test_file}")

Encoded test data saved to /content/encoded_chunks/encoded_test_data.csv
Saving to: /content/encoded_chunks/encoded_test_data.csv


In [12]:
# train인코딩

# 청크 파일이 저장된 디렉토리 경로
directory_path =  '/content/encoded_chunks/'
chunksize = 100000

# 청크 파일 리스트 가져오기
encoded_chunk_files = sorted(glob.glob(directory_path + 'encoded_chunk_*.csv'))


# 각 청크 파일을 읽어서 데이터 타입 확인
for file in encoded_chunk_files:
    print(f"Processing file: {file}")

    # 청크 파일 읽기
    chunk_generator = pd.read_csv(file, chunksize=chunksize)

    # 각 청크에 대해 데이터 타입 확인
    for i, chunk_part in enumerate(chunk_generator):
        print(f"Chunk {i+1} of {file}:")
        print("Data Types:")
        print(chunk_part.dtypes)  # 각 열의 데이터 타입 출력

        # 수치형 데이터 타입 확인
        numeric_cols = chunk_part.select_dtypes(include=['number']).columns
        print("\nNumeric Columns:")
        print(numeric_cols)  # 수치형 열 목록 출력

        print()  # 빈 줄로 구분

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
F18      float64
F19      float64
F20        int64
F21        int64
F22        int64
F23        int64
F24      float64
F25        int64
F26        int64
F27      float64
F28        int64
F29      float64
F30        int64
F31        int64
F32      float64
F33      float64
F34        int64
F35        int64
F36      float64
F37        int64
F38      float64
F39        int64
dtype: object

Numeric Columns:
Index(['ID', 'Click', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18',
       'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39'],
      dtype='object')

Processing file: /content/encoded_chunks/encoded_chunk_275.csv
Chunk 1 of /content/encoded_chunks/encoded_chunk_275.csv:
Data Types:
ID         int64
Click      int64
F01        int64
F02        int64
F03       

In [13]:
#인코딩 확인
encoded_test_data = pd.read_csv('/content/encoded_chunks/encoded_test_data.csv')
encoded_test_data.dtypes

,0
ID,float64
F01,float64
F02,float64
F03,float64
F04,float64
F05,float64
F06,int64
F07,float64
F08,float64
F09,float64


## High cardinality 컬럼

In [17]:
# 청크 파일이 저장된 디렉토리 경로
directory_path = '/content/encoded_chunks/'
chunksize = 100000

# 청크 파일 리스트 가져오기
chunk_files = sorted(glob.glob(directory_path + 'encoded_chunk_*.csv'))

def calculate_nunique_chunks(chunk_files, chunksize=50000):
    chunk_iter = pd.read_csv(chunk_files, chunksize=chunksize)
    unique_counts = {}
    for chunk in chunk_iter:
        for col in chunk.select_dtypes(include='object').columns:
            if col not in unique_counts:
                unique_counts[col] = set()
            unique_counts[col].update(chunk[col].unique())

    unique_counts = {col: len(unique_set) for col, unique_set in unique_counts.items()}
    return unique_counts

if __name__ == "__main__":
    # Google Drive 경로를 확인하여 정확하게 입력
    train_path = '/content/drive/MyDrive/zerobase/광고클릭률예측/open/train.csv'
    nunique_counts = calculate_nunique_chunks(train_path)

    # 딕셔너리를 DataFrame으로 변환하고 정렬
    nunique_df = pd.DataFrame(list(nunique_counts.items()), columns=['Column', 'Unique Count'])
    nunique_df_sorted = nunique_df.sort_values(by='Unique Count', ascending=False)

    # 정렬된 DataFrame 출력
    print(nunique_df_sorted)

   Column  Unique Count
0      ID      28605391
4     F05       5343557
1     F01       4760931
9     F12       4174064
23    F34       3165581
8     F10       1404255
2     F02        304405
14    F20        178603
5     F07        151200
7     F09         27551
21    F30         19444
12    F16         15467
18    F25         10700
25    F37          9423
16    F22          7187
26    F39          6800
19    F26          2205
10    F13          1307
17    F23           950
6     F08            79
3     F03            64
20    F28            55
15    F21            33
22    F31            14
13    F17            10
11    F15             4
24    F35             3


   F05       5343557 </br>
   F01       4760931 </br>
   F12       4174064 </br>
   F34       3165581 </br>
   F10       1404255 </br>
High Cardinal한 컬럼을 제거해서 메모리 사용량 축소 </br>
: 모델이 high cardinality 컬럼의 정보를 잘 활용하지 못하거나, 성능 향상에 기여하지 않는 경우가 많습니다.  </br>이러한 경우, 불필요한 컬럼을 제거함으로써 모델의 학습 시간과 자원 소모를 줄일 수 있기 때문에.

# 1차 모델학습

In [9]:
import glob

# 청크 파일이 저장된 디렉토리 경로
directory_path = '/content/encoded_chunks/'

# 디렉토리 내에서 특정 패턴에 맞는 모든 파일 경로를 가져오기
chunk_files = sorted(glob.glob(directory_path + 'encoded_chunk_*.csv'))

# 가져온 파일 리스트 출력
print(f"Found {len(chunk_files)} chunk files.")

Found 287 chunk files.


In [10]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# 청크 파일의 첫 번째 파일을 로드하여 전체 데이터를 준비합니다.
encoded_dir = '/content/encoded_chunks/'
encoded_chunk_files = sorted(glob.glob(encoded_dir + 'encoded_chunk_*.csv'))

# 전체 데이터를 준비합니다.
# 예시로 첫 번째 청크 파일을 사용합니다.
data = pd.read_csv(encoded_chunk_files[0])
X = data.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
y = data['Click']

# 훈련 데이터와 검증 데이터로 분할합니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
eval_set = [(X_val, y_val)]  # 검증 데이터 설정

# CatBoost 모델 초기화
model = CatBoostClassifier(
    iterations=1000,  # 원하는 반복 수 설정
    depth=6,          # 트리 깊이
    learning_rate=0.1,
    loss_function='Logloss',
    task_type='CPU',  # CPU 사용
    verbose=1000,     # 학습 과정 출력
    early_stopping_rounds=10  # 초기 학습 시 얼리스탑어라운드 설정
)

# 초기 학습
train_pool = Pool(X_train, label=y_train)
model.fit(train_pool, eval_set=eval_set, early_stopping_rounds=10)

# 모델 업데이트
for file in encoded_chunk_files[1:]:
    chunk = pd.read_csv(file)
    X_train = chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
    y_train = chunk['Click']

    train_pool = Pool(X_train, label=y_train)
    model.fit(train_pool, init_model=model, verbose=1000, eval_set=eval_set, early_stopping_rounds=10)

# 학습 완료된 모델 저장
model.save_model('/content/catboost_model.cbm')

0:	learn: 0.6433851	test: 0.6434654	best: 0.6434654 (0)	total: 108ms	remaining: 1m 47s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.4175109923
bestIteration = 308

Shrink model to first 309 iterations.
0:	learn: 0.4475766	test: 0.4174535	best: 0.4174535 (0)	total: 66.2ms	remaining: 1m 6s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.4172969354
bestIteration = 5

Shrink model to first 6 iterations.
0:	learn: 0.4562919	test: 0.4173057	best: 0.4173057 (0)	total: 62.7ms	remaining: 1m 2s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.4169059116
bestIteration = 19

Shrink model to first 20 iterations.
0:	learn: 0.4566533	test: 0.4169213	best: 0.4169213 (0)	total: 55.5ms	remaining: 55.4s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.416921348
bestIteration = 0

Shrink model to first 1 iterations.
0:	learn: 0.4558494	test: 0.4169601	best: 0.4169601 (0)	total: 50.6ms	remaining: 50.5s
Stopped by overfitting d

##  feature_importances

In [17]:
from catboost import CatBoostClassifier

label = 'Click'
categorical_columns = [
    'F02', 'F03', 'F07', 'F08', 'F09', 'F13', 'F15', 'F16', 'F17',
    'F20', 'F21', 'F22', 'F23', 'F25', 'F26', 'F28', 'F30', 'F31',
    'F35', 'F37', 'F39'
]

best_model_path = '/content/catboost_model.cbm'
model = CatBoostClassifier()
model.load_model(best_model_path)

feature_importances = model.get_feature_importance(type='FeatureImportance')
feature_names = model.feature_names_

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

   Feature  Importance
4      F07    5.430621
6      F09    4.931369
16     F21    4.471226
8      F13    4.349468
0      F02    4.276111
3      F06    4.113677
27     F32    3.918145
31     F37    3.884583
20     F25    3.537387
26     F31    3.537180
23     F28    3.412234
7      F11    3.411378
33     F39    3.406357
2      F04    3.331293
24     F29    3.319156
11     F16    3.207146
5      F08    3.169901
21     F26    2.992534
19     F24    2.989026
10     F15    2.755503
12     F17    2.633545
22     F27    2.463786
28     F33    2.271102
1      F03    2.185275
15     F20    2.178459
14     F19    2.045781
17     F22    1.878657
30     F36    1.782823
9      F14    1.695817
13     F18    1.525557
25     F30    1.328667
32     F38    1.327660
29     F35    1.169004
18     F23    1.069572


In [17]:
from sklearn.metrics import roc_auc_score

In [18]:
from catboost import CatBoostClassifier
from sklearn.metrics import precision_score, recall_score, f1_score,classification_report,confusion_matrix
import pandas as pd
import glob

# 청크 파일이 저장된 디렉토리 경로
directory_path =  '/content/encoded_chunks/'
chunksize = 100000

# 청크 파일 리스트 가져오기
encoded_chunk_files = sorted(glob.glob(directory_path + 'encoded_chunk_*.csv'))

# 모델 로드
#model = CatBoostClassifier()
model.load_model('/content/catboost_model.cbm')

y_true = []
y_pred = []

# 청크 파일을 순회하며 예측
for file in encoded_chunk_files:
    print(f"Processing file: {file}")
    chunk = pd.read_csv(file)

    # 데이터가 비어 있는지 확인
    if chunk.empty:
        print(f"Warning: The file {file} is empty.")
        continue

    # 데이터 준비
    X_chunk = chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1, errors='ignore')
    y_chunk = chunk.get('Click')

    if X_chunk.empty:
        print(f"Warning: No features in file {file}.")
        continue

    if y_chunk is None:
        print(f"Warning: 'Click' column missing in file {file}.")
        continue

    # **데이터가 범주형 열을 올바르게 처리하도록 설정**
    #for col in categorical_columns:
     #   if col in X_chunk.columns:
      #      X_chunk[col] = X_chunk[col].astype('category')
    # 예측
    try:
        y_true.extend(y_chunk.tolist())
        y_pred.extend(model.predict(X_chunk).tolist())
    except Exception as e:
        print(f"Error during prediction for file {file}: {e}")


Processing file: /content/encoded_chunks/encoded_chunk_1.csv
Processing file: /content/encoded_chunks/encoded_chunk_10.csv
Processing file: /content/encoded_chunks/encoded_chunk_100.csv
Processing file: /content/encoded_chunks/encoded_chunk_101.csv
Processing file: /content/encoded_chunks/encoded_chunk_102.csv
Processing file: /content/encoded_chunks/encoded_chunk_103.csv
Processing file: /content/encoded_chunks/encoded_chunk_104.csv
Processing file: /content/encoded_chunks/encoded_chunk_105.csv
Processing file: /content/encoded_chunks/encoded_chunk_106.csv
Processing file: /content/encoded_chunks/encoded_chunk_107.csv
Processing file: /content/encoded_chunks/encoded_chunk_108.csv
Processing file: /content/encoded_chunks/encoded_chunk_109.csv
Processing file: /content/encoded_chunks/encoded_chunk_11.csv
Processing file: /content/encoded_chunks/encoded_chunk_110.csv
Processing file: /content/encoded_chunks/encoded_chunk_111.csv
Processing file: /content/encoded_chunks/encoded_chunk_112.

In [18]:
# 평가 지표 출력
if y_true and y_pred:
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, zero_division=0))
    print("Recall:", recall_score(y_true, y_pred, zero_division=0))
    print("F1 Score:", f1_score(y_true, y_pred, zero_division=0))
    print("ROC AUC Score:", roc_auc_score(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.8127314183539739
Precision: 0.5801578549195106
Recall: 0.13837008470589926
F1 Score: 0.22344709815976269
ROC AUC Score: 0.5570791134729078
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.98      0.89  23035531
           1       0.58      0.14      0.22   5569860

    accuracy                           0.81  28605391
   macro avg       0.70      0.56      0.56  28605391
weighted avg       0.78      0.81      0.76  28605391

Confusion Matrix:
 [[22477798   557733]
 [ 4799158   770702]]


**데이터 밸런스 조정**</br>
정확도는 높으나 정밀도와 재현율이 낮은걸로 보아 </br>
클릭한 광고(positive) 데이터의 비율이 너무 낮을 수 있어서 데이터의 불균형이 </br>일어났을 것이라 가정하고 scale_pos_weight로 데이터 불균형을 해소</br>

**모델 하이퍼파라미터 튜닝**</br>
CatBoost의 Optuna 라이브러리를 통해 하이퍼파라미터 최적화

## scale_pos_weight

In [22]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

label = 'Click'
categorical_features =  [ 'ID','F02', 'F03', 'F07', 'F08', 'F09', 'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F25', 'F26', 'F28', 'F31', 'F35', 'F37', 'F39']

for file in encoded_chunk_files:
    print(f"Processing file: {file}")
    chunk = pd.read_csv(file)

    num_positive = chunk[chunk[label] == 1].shape[0]
    num_negative = chunk[chunk[label] == 0].shape[0]
    scale_pos_weight = num_negative / num_positive

    train_pool = Pool(data=chunk.drop(columns=[label]), label=chunk[label],
                                cat_features=categorical_features
                                )
    val_pool = Pool(data=chunk.drop(columns=[label]), label=chunk[label]
                                , cat_features=categorical_features
                                )

print(f"weight: {scale_pos_weight}")

Processing file: /content/encoded_chunks/encoded_chunk_1.csv
Processing file: /content/encoded_chunks/encoded_chunk_10.csv
Processing file: /content/encoded_chunks/encoded_chunk_100.csv
Processing file: /content/encoded_chunks/encoded_chunk_101.csv
Processing file: /content/encoded_chunks/encoded_chunk_102.csv
Processing file: /content/encoded_chunks/encoded_chunk_103.csv
Processing file: /content/encoded_chunks/encoded_chunk_104.csv
Processing file: /content/encoded_chunks/encoded_chunk_105.csv
Processing file: /content/encoded_chunks/encoded_chunk_106.csv
Processing file: /content/encoded_chunks/encoded_chunk_107.csv
Processing file: /content/encoded_chunks/encoded_chunk_108.csv
Processing file: /content/encoded_chunks/encoded_chunk_109.csv
Processing file: /content/encoded_chunks/encoded_chunk_11.csv
Processing file: /content/encoded_chunks/encoded_chunk_110.csv
Processing file: /content/encoded_chunks/encoded_chunk_111.csv
Processing file: /content/encoded_chunks/encoded_chunk_112.

## Optuna 라이브러리로 하이퍼파라미터 최적화

In [24]:
import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import pandas as pd
import glob

# 청크 데이터 파일 경로
encoded_dir = '/content/encoded_chunks/'
encoded_chunk_files = sorted(glob.glob(encoded_dir + 'encoded_chunk_*.csv'))

# 검증 데이터 준비 (여기서는 첫 번째 청크를 검증 데이터로 사용한다고 가정)
val_chunk = pd.read_csv(encoded_chunk_files[0])
X_val = val_chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
y_val = val_chunk['Click']

# 카테고리형 특성 리스트
categorical_features = ['F02', 'F03', 'F07', 'F08', 'F09', 'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F25', 'F26', 'F28', 'F31', 'F35', 'F37', 'F39']

# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    param = {
        'iterations': trial.suggest_int('iterations', 500, 1000),  # iterations 범위 조정
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 1e2, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'border_count': trial.suggest_int('border_count', 1, 255),
    }

    # 모델 초기화
    model = CatBoostClassifier(**param, cat_features=categorical_features, verbose=100)

    # 청크 데이터를 사용하여 점진적으로 모델 학습
    for file in encoded_chunk_files[1:]:
        chunk = pd.read_csv(file)
        X_train = chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
        y_train = chunk['Click']

        # Pool 객체 생성
        train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)

        # 초기화된 모델에 대한 점진적 학습
        model.fit(train_pool,
                  verbose=0)

    # 마지막 청크 학습 후 검증 데이터로 AUC 계산
    y_pred = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred)

    return auc

# Optuna 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best AUC score:", study.best_value)
print("Best parameters:", study.best_params)

# 학습 완료된 모델 저장
model.save_model('/content/catboost_model.cbm')

[I 2024-08-29 08:02:07,834] A new study created in memory with name: no-name-42014753-6f49-4f32-b806-ee4d696eee2a
[W 2024-08-29 08:06:06,267] Trial 0 failed with parameters: {'iterations': 504, 'depth': 9, 'learning_rate': 6.709016140452859e-05, 'l2_leaf_reg': 0.8861800865870589, 'random_strength': 2.3069234814853967, 'bagging_temperature': 3.7326432076849034, 'border_count': 133} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-24-1916eb8b3cd2>", line 45, in objective
    model.fit(train_pool,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5220, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2400,

KeyboardInterrupt: 

In [11]:
import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import pandas as pd
import glob

# 청크 데이터 파일 경로
encoded_dir = '/content/encoded_chunks/'
encoded_chunk_files = sorted(glob.glob(encoded_dir + 'encoded_chunk_*.csv'))

# 카테고리형 특성 리스트 (예시로 수정 필요)
#categorical_features = ['F02', 'F03', 'F07', 'F08', 'F09', 'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F25', 'F26', 'F28', 'F31', 'F35', 'F37', 'F39']

# 검증 데이터 준비 (여기서는 첫 번째 청크를 검증 데이터로 사용한다고 가정)
val_chunk = pd.read_csv(encoded_chunk_files[0])
X_val = val_chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
y_val = val_chunk['Click']
eval_set = [(X_val, y_val)]  # 검증 데이터 설정

# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    param = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 1e2, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'border_count': trial.suggest_int('border_count', 1, 255),
    }

    # 모델 초기화
    model = CatBoostClassifier(**param, cat_features=categorical_features, verbose=100)

    # 청크 데이터를 사용하여 점진적으로 모델 학습
    for file in encoded_chunk_files[1:]:
        chunk = pd.read_csv(file)
        X_train = chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
        y_train = chunk['Click']

        # Pool 객체 생성
        train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)

        # 모델을 점진적으로 업데이트
        #model.fit(train_pool, init_model=model if file != encoded_chunk_files[1] else None, verbose=0)
        model.fit(train_pool,
                 # init_model=model,
                  verbose=1000,
                  eval_set=eval_set,
                  early_stopping_rounds=10)

    # 검증 데이터로 AUC 계산
    val_pool = Pool(X_val, label=y_val, cat_features=categorical_features)
    y_pred = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred)

    return auc

# 학습 완료된 모델 저장
model.save_model('/content/catboost_model.cbm')

# Optuna 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best AUC score:", study.best_value)
print("Best parameters:", study.best_params)


CatBoostError: There is no trained model to use save_model(). Use fit() to train model. Then use this method.

In [11]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    param = {
         'iterations': 5000,
         'depth': 10,
        'learning_rate': 0.110521996103408,
        'border_count': 255,
        'eval_metric': 'AUC',
        'verbose': True,
        'task_type': 'CPU',
        'devices': '0',
        'l2_leaf_reg': 3.322521996103408,
        'scale_pos_weight': 3.967463116586359,
        'bagging_temperature': 0.1277658067340062,
        'metric_period': 100,
        'save_snapshot': True,
        'snapshot_file': 'catboost_snapshot',
        'random_strength': 5,
        'snapshot_interval': 1200,
        'use_best_model': True,
        'model_size_reg': 1,
        'boosting_type': 'Plain'
    }

    # 모델 학습
    model = CatBoostClassifier(**param, cat_features=categorical_features, verbose=0)
    model.fit(X_train, y_train)

    # 검증 데이터로 AUC 계산
    y_pred = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred)

    return auc

In [12]:
# Optuna 최적화 실행
import optuna

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best AUC score:", study.best_value)
print("Best parameters:", study.best_params)

[I 2024-08-30 05:15:11,893] A new study created in memory with name: no-name-d66dbcd1-30d3-460a-b82f-ce19aa064169
[W 2024-08-30 05:15:11,904] Trial 0 failed with parameters: {} because of the following error: NameError("name 'categorical_features' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-11-4964ac165c38>", line 30, in objective
    model = CatBoostClassifier(**param, cat_features=categorical_features, verbose=0)
NameError: name 'categorical_features' is not defined
[W 2024-08-30 05:15:11,913] Trial 0 failed with value None.


NameError: name 'categorical_features' is not defined

# 모델 하이퍼파라미터 적용

In [13]:
# 청크 파일의 첫 번째 파일을 로드하여 전체 데이터를 준비합니다.
encoded_dir = '/content/encoded_chunks/'
encoded_chunk_files = sorted(glob.glob(encoded_dir + 'encoded_chunk_*.csv'))

# 전체 데이터를 준비합니다.
# 예시로 첫 번째 청크 파일을 사용합니다.
data = pd.read_csv(encoded_chunk_files[0])
X = data.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
y = data['Click']

# 훈련 데이터와 검증 데이터로 분할합니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
eval_set = [(X_val, y_val)]  # 검증 데이터 설정


best_params = {
    'iterations': 716,
    'depth': 6,
    'learning_rate': 0.1434452681311719,
    'random_strength': 9.636924231305613,
    'bagging_temperature': 0.8801310826837648,
    'border_count': 181,
    'l2_leaf_reg': 4.3754592812493405,
    'scale_pos_weight': 0.5297966866606383,
    'eval_metric': 'AUC',
    'task_type': 'CPU',
    'random_seed': 42
}

# CatBoost 모델 초기화
model = CatBoostClassifier(
    **best_params,
   # iterations=1000,  # 원하는 반복 수 설정
    #depth=6,          # 트리 깊이
    #learning_rate=0.1,
    #loss_function='Logloss',
    #task_type='CPU',  # CPU 사용
    #verbose=1000,     # 학습 과정 출력
    #early_stopping_rounds=10  # 초기 학습 시 얼리스탑어라운드 설정
)

# 초기 학습
train_pool = Pool(X_train, label=y_train)
model.fit(train_pool, eval_set=eval_set, early_stopping_rounds=10)

# 모델 업데이트
for file in encoded_chunk_files[1:]:
    chunk = pd.read_csv(file)
    X_train = chunk.drop(['Click', 'ID', 'F01', 'F05', 'F10', 'F12', 'F34'], axis=1)
    y_train = chunk['Click']

    train_pool = Pool(X_train, label=y_train)
    model.fit(train_pool,
              #init_model=model,
              verbose=1000, eval_set=eval_set, early_stopping_rounds=500,save_snapshot=False)

# 학습 완료된 모델 저장
model.save_model('/content/catboost_model.cbm')

0:	test: 0.5214449	best: 0.5214449 (0)	total: 63.4ms	remaining: 45.3s
1:	test: 0.5342830	best: 0.5342830 (1)	total: 110ms	remaining: 39.4s
2:	test: 0.5494670	best: 0.5494670 (2)	total: 162ms	remaining: 38.5s
3:	test: 0.5687814	best: 0.5687814 (3)	total: 209ms	remaining: 37.2s
4:	test: 0.5768958	best: 0.5768958 (4)	total: 285ms	remaining: 40.5s
5:	test: 0.5954273	best: 0.5954273 (5)	total: 340ms	remaining: 40.3s
6:	test: 0.6255321	best: 0.6255321 (6)	total: 399ms	remaining: 40.4s
7:	test: 0.6263826	best: 0.6263826 (7)	total: 450ms	remaining: 39.8s
8:	test: 0.6324460	best: 0.6324460 (8)	total: 504ms	remaining: 39.6s
9:	test: 0.6497678	best: 0.6497678 (9)	total: 556ms	remaining: 39.2s
10:	test: 0.6591209	best: 0.6591209 (10)	total: 608ms	remaining: 39s
11:	test: 0.6625887	best: 0.6625887 (11)	total: 657ms	remaining: 38.6s
12:	test: 0.6633428	best: 0.6633428 (12)	total: 725ms	remaining: 39.2s
13:	test: 0.6678323	best: 0.6678323 (13)	total: 796ms	remaining: 39.9s
14:	test: 0.6683774	best: 0

In [23]:
catboost_params = {
    'iterations': 5000,
    'depth': 10,
    'learning_rate': 0.110521996103408,
    'border_count': 255,
    'eval_metric': 'AUC',
    'verbose': True,
    'task_type': 'CPU',
    'devices': '0',
    'l2_leaf_reg': 3.322521996103408,
    'scale_pos_weight': 3.967463116586359,
    'bagging_temperature': 0.1277658067340062,
    'metric_period': 100,
    'save_snapshot': True,
    'snapshot_file': 'catboost_snapshot',
    'random_strength': 5,
    'snapshot_interval': 1200,
    'use_best_model': True,
    'model_size_reg': 1,
    'boosting_type': 'Plain'
}

# X_val에서 범주형 피처의 인덱스를 지정
categorical_features = ['ID','F02', 'F03', 'F07', 'F08', 'F09', 'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F25', 'F26', 'F28', 'F31', 'F35', 'F37', 'F39']


val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_features)


model = CatBoostClassifier(**catboost_params)

print("훈련 개시")
model.fit(train_pool
          , eval_set=val_pool
          , use_best_model=True
          , early_stopping_rounds=50
          )
print("종료")

ValueError: 'ID' is not in list

## 테스트 데이터 적용

In [7]:

# 테스트 데이터 파일 경로
test_file_path = '/content/encoded_chunks/encoded_test_data.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

catboost_params = {
    'iterations': 5000,
    'depth': 10,
    'learning_rate': 0.110521996103408,
    'border_count': 255,
    'eval_metric': 'AUC',
    'verbose': True,
    'task_type': 'CPU',
    'devices': '0',
    'l2_leaf_reg': 3.322521996103408,
    'scale_pos_weight': 3.967463116586359,
    'bagging_temperature': 0.1277658067340062,
    'metric_period': 100,
    'save_snapshot': True,
    'snapshot_file': 'catboost_snapshot',
    'random_strength': 5,
    'snapshot_interval': 1200,
    'use_best_model': True,
    'model_size_reg': 1,
    'boosting_type': 'Plain'
}

model = CatBoostClassifier(**catboost_params)

# 테스트 데이터에 대한 예측
y_test_pred_proba = model.predict_proba(test_data)[:, 1]  # 예측 확률 (Positive Class)
y_test_pred = model.predict(test_data)  # 예측 결과 (0 or 1)

# 예측 결과를 테스트 데이터에 추가
test_data['Predicted_Click'] = y_test_pred

# 결과 저장
test_data.to_csv('test_data_with_predictions.csv', index=False)
print("Test data with predictions saved.")



CatBoostError: There is no trained model to use predict_proba(). Use fit() to train model. Then use this method.

In [ ]:
model.save_model('catboost_model.cbm')
print("모델 저장")

In [ ]:
test_data = pd.read_csv("test_final.csv")

# 문제 발생했을 경우 모델 로드
#model = CatBoostClassifier()
#model.load_model("catboost_model.cbm")
#categorical_features = ['F02', 'F03', 'F07', 'F08', 'F09', 'F13', 'F15', 'F16', 'F17', 'F20', 'F21', 'F25', 'F26', 'F28', 'F31', 'F35', 'F37', 'F39']

test_pool = Pool(data=test_data.drop(columns=['ID']), cat_features=categorical_features)
predictions = model.predict_proba(test_pool)[:, 1]
submission = pd.DataFrame({
    'ID': test_data['ID'],
    'Click': predictions
})
submission.to_csv("submission.csv", index=False)
print("저장 ")